In [1]:
import numpy as np
import tensorflow as tf
import pandas as analytics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-04-13 18:04:00.228017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 18:04:00.230138: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 18:04:00.297566: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 18:04:00.578171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 18:04:01.432698: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
df_corpus = analytics.read_csv('../data/ptb.train.txt', header = None)
corpus = []
for i in range(len(df_corpus[:5000])):
    text = df_corpus.iloc[i][0].strip()
    corpus.append(text)

In [3]:
# Example corpus
# corpus = [
#     "This is a simple example sentence.",
#     "Another example sentence for training."
# ]

# Tokenize the corpus
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Generate input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
X, y = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Define the model architecture
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

/home/ashirbad/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-04-13 18:04:03.834480: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 133s 42ms/step - accuracy: 0.0748 - loss: 6.7744
Epoch 2/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 134s 42ms/step - accuracy: 0.1408 - loss: 5.8113
Epoch 3/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 135s 43ms/step - accuracy: 0.1728 - loss: 5.3263
Epoch 4/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 135s 43ms/step - accuracy: 0.1954 - loss: 4.9337
Epoch 5/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 136s 43ms/step - accuracy: 0.2190 - loss: 4.5563
Epoch 6/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 136s 43ms/step - accuracy: 0.2396 - loss: 4.2273
Epoch 7/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 136s 43ms/step - accuracy: 0.2698 - loss: 3.8843
Epoch 8/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 136s 43ms/step - accuracy: 0.3009 - loss: 3.6049
Epoch 9/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 137s 43ms/step - accuracy: 0.3377 - loss: 3.3181
Epoch 10/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 137s 43ms/step - accuracy: 0.3785 - loss: 3.0556
Epoch 11/100
3161/3161 ━━━━━━━━━━━━━━━━━━━━ 137s 43ms/step - accuracy: 0.4157 -

In [4]:
tf.saved_model.save(model,"trained_model.keras")

INFO:tensorflow:Assets written to: trained_model.keras/assets


INFO:tensorflow:Assets written to: trained_model.keras/assets


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 97, 100)        │       697,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6977)           │     1,053,527 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,705,483 (21.76 MB)

 Trainable params: 1,901,827 (7.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,803,656 (14.51 MB)

In [6]:
# model = tf.keras.models.load_model('trained_model.keras')

In [7]:
# Function to generate text
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        # Tokenize the seed text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # Pad the token list
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # Predict the next word
        predicted_probs = model.predict(token_list, verbose=0)[0]
        # Sample the next word based on the predicted probabilities
        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        # Convert the predicted word index back to the actual word
        predicted_word = tokenizer.index_word.get(predicted_word_index, "")
        # Update the seed text with the predicted word
        seed_text += " " + predicted_word
    return seed_text

# Example usage
# seed_text = "nonexecutive director"
# generated_text = generate_text(seed_text, next_words=10, max_sequence_len=max_sequence_len)
# print("Generated Text:", generated_text)


In [ ]:
stop_word = "Thank You !"
input_text = "Welcome"
print("Welcome to the chatbot !")
while input_text != stop_word :
    print("\n",generate_text(input_text, next_words=10, max_sequence_len=max_sequence_len))
    print("\n")
    input_text = input("Enter your response : ")

print("\n\n\nIt was great talking with you !!")

Welcome to the chatbot !

 Welcome the unk construction practices and unk workout of the company




Enter your response :  yellow



 yellow first financial corp said its unk ships are n't expected




Enter your response :  color



 color company said it was a registration to increase n and


